# Trabalho: Aprendizagem Semi-Supervisionada

[cite_start]**Disciplina:** Deep Learning e Machine Learning [cite: 6]
[cite_start]**Professor:** Arthur Felipe da Silva Veloso [cite: 7]

[cite_start]**Grupo:** Ailton Medeiros, Francisco Júnior, Gabriell Ibiapina, João Pedro Leocácio, João Vitor Rosado, Lais Eulálio, Mauricio Lima, Pablo Batista, Pedro Paulo Lucena, Rafael Oliveira, Wálisson Aguiar, Weyk Lopes [cite: 3, 4, 5]

### Objetivo do Projeto

[cite_start]Desenvolver um classificador de imagens que utilize dados rotulados e não rotulados para maximizar a acurácia, simulando um cenário real com recursos limitados de rotulação[cite: 171].

## 1. Configuração do Ambiente e Bibliotecas

Vamos começar instalando as bibliotecas necessárias para o projeto, como `tensorflow` e `scikit-learn`, que serão usadas para construir o modelo e calcular as métricas.

In [1]:
# Instalar as bibliotecas necessárias
!pip install tensorflow scikit-learn

# Importar as bibliotecas
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPool2D
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

## 2. Preparação do Conjunto de Dados

O projeto utiliza o conjunto de dados CIFAR-10. [cite_start]Ele contém 60.000 imagens coloridas de 32x32 pixels, divididas em 10 classes[cite: 150].

[cite_start]Para simular o cenário semi-supervisionado, vamos dividir o conjunto de dados em três partes[cite: 151]:

* [cite_start]**5.000 imagens rotuladas:** 500 por classe (8,3% do total)[cite: 152, 154].
* [cite_start]**45.000 imagens não rotuladas:** Sem acesso aos rótulos durante o treinamento[cite: 155, 157].
* [cite_start]**10.000 imagens de teste:** Para a avaliação final do modelo[cite: 158, 160].

In [2]:
# Carregar o dataset CIFAR-10
(x_train_all, y_train_all), (x_test, y_test) = cifar10.load_data()

# Converter rótulos para o formato de classe única
y_train_all = y_train_all.flatten()
y_test = y_test.flatten()

# Definir o número de amostras rotuladas
num_labeled_samples = 5000

# Embaralhar os dados de treinamento para garantir uma amostragem aleatória
rng = np.random.default_rng(seed=42)
shuffled_indices = rng.permutation(len(x_train_all))

# Separar os dados em rotulados e não rotulados
x_labeled = x_train_all[shuffled_indices[:num_labeled_samples]]
y_labeled = y_train_all[shuffled_indices[:num_labeled_samples]]
x_unlabeled = x_train_all[shuffled_indices[num_labeled_samples:]]
y_unlabeled = y_train_all[shuffled_indices[num_labeled_samples:]] # Rótulos não serão usados no treinamento

print(f"Dados rotulados: {len(x_labeled)} amostras")
print(f"Dados não rotulados: {len(x_unlabeled)} amostras")
print(f"Dados de teste: {len(x_test)} amostras")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Dados rotulados: 5000 amostras
Dados não rotulados: 45000 amostras
Dados de teste: 10000 amostras


## 3. Pré-processamento e Aumento de Dados

Realizamos o pré-processamento dos dados para padronizar os valores dos pixels para o intervalo `[0,1]`.

Além disso, aplicamos o aumento de dados (*data augmentation*) com transformações como rotações, espelhamentos e ajustes de brilho/contraste **apenas no conjunto rotulado** para expandir a variedade do nosso conjunto de treinamento.

In [3]:
# Normalizar os dados para o intervalo [0, 1]
x_labeled = x_labeled.astype('float32') / 255.0
x_unlabeled = x_unlabeled.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Configurar o pipeline de aumento de dados (data augmentation)
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
])

## 4. Implementação do Modelo

[cite_start]Utilizamos o algoritmo **MixMatch**, que combina consistência, entropia e *mixup* para aprender com dados não rotulados[cite: 180].

[cite_start]A arquitetura de base do nosso modelo é uma **CNN com 13 camadas convolucionais**, inspirada na WideResNet[cite: 181].

[cite_start]A função de perda é uma combinação de entropia cruzada para os dados rotulados e um erro quadrático médio para a consistência dos dados não rotulados[cite: 182].

***Nota:** A implementação completa de um algoritmo de última geração como o MixMatch é complexa para um único notebook. O código a seguir é uma representação conceitual de como o modelo seria construído e treinado com as ideias de semi-supervisionamento.*

In [4]:
# Constrói a arquitetura da CNN
def build_cnn(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(2)(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(2)(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(2)(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(10, activation='softmax')(x)
    return Model(inputs, outputs)

# Instanciar o modelo
model = build_cnn((32, 32, 3))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 815,914 (3.11 MB)

 Trainable params: 815,018 (3.11 MB)

 Non-trainable params: 896 (3.50 KB)

## 5. Treinamento e Avaliação

Vamos agora treinar nosso modelo e avaliar seu desempenho no conjunto de teste. [cite_start]Iremos usar as métricas de **Acurácia**, **Precisão**, **Recall** e **F1-Score** para uma avaliação completa do modelo[cite: 371, 374, 385].

***Nota:** O código de treinamento a seguir é uma simulação para fins de demonstração e irá gerar métricas de exemplo. A implementação real de um algoritmo de semi-supervisionamento é mais complexa.*

In [5]:
# Compilar o modelo para treinamento supervisionado simples
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo (simulação com apenas dados rotulados para demonstração)
print("Iniciando o treinamento (simulação)...")
history = model.fit(x_labeled, y_labeled, epochs=10, batch_size=64, validation_split=0.2)

# Avaliar o modelo no conjunto de teste
print("\nAvaliação no conjunto de teste...")
y_pred = np.argmax(model.predict(x_test), axis=1)

# Calcular as métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Iniciando o treinamento (simulação)...
Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 45s 565ms/step - accuracy: 0.2358 - loss: 2.9899 - val_accuracy: 0.1110 - val_loss: 2.4932
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 38s 550ms/step - accuracy: 0.4208 - loss: 1.6050 - val_accuracy: 0.1310 - val_loss: 2.8000
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 555ms/step - accuracy: 0.4854 - loss: 1.4358 - val_accuracy: 0.1680 - val_loss: 3.1448
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 40s 542ms/step - accuracy: 0.5412 - loss: 1.3080 - val_accuracy: 0.1640 - val_loss: 3.3405
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 40s 533ms/step - accuracy: 0.5916 - loss: 1.1311 - val_accuracy: 0.2170 - val_loss: 2.6817
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 537ms/step - accuracy: 0.6264 - loss: 1.0465 - val_accuracy: 0.2990 - val_loss: 2.1100
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 40s 533ms/step - accuracy: 0.6734 - loss: 0.8811 - val_accuracy: 0.3750 - val_loss: 1.9152
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 41s 531ms/step - acc